## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-02-12-55-32 +0000,nyt,"Meet Your News Friend, the New Host of The Mor...",https://www.nytimes.com/2025/11/02/world/02the...
1,2025-11-02-12-54-37 +0000,nyt,The New Host of The Morning,https://www.nytimes.com/2025/11/02/world/sam-s...
2,2025-11-02-12-51-21 +0000,nyt,War Powers Law Does Not Apply to Trump’s Boat ...,https://www.nytimes.com/2025/11/01/us/politics...
3,2025-11-02-12-39-00 +0000,bbc,Trump tells military to prepare for 'action' a...,https://www.bbc.com/news/articles/cev18jy21w7o...
4,2025-11-02-12-07-04 +0000,nyt,Trump’s Retribution Campaign Leaves D.C. Prose...,https://www.nytimes.com/2025/11/02/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,26
66,train,8
4,new,8
207,up,8
202,dead,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
114,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...,56
13,2025-11-02-11-26-41 +0000,nypost,White House unveils details of ‘historic’ trad...,https://nypost.com/2025/11/02/us-news/donald-t...,52
90,2025-11-02-00-19-33 +0000,wapo,Trump threatens potential military action in N...,https://www.washingtonpost.com/world/2025/11/0...,52
141,2025-11-01-18-25-24 +0000,nypost,Nicki Minaj sends lefty fans into meltdown aft...,https://nypost.com/2025/11/01/us-news/nicki-mi...,51
67,2025-11-02-03-49-34 +0000,bbc,New Yorkers could pick a political newcomer to...,https://www.bbc.com/news/articles/c3rjjdvx5r5o...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
114,56,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...
119,39,2025-11-01-21-04-33 +0000,nypost,NJ Dem chair admits ‘New Jersey is a battlegro...,https://nypost.com/2025/11/01/us-news/nj-dem-b...
120,37,2025-11-01-20-55-31 +0000,nypost,House Republicans exploring ways to prevent Ma...,https://nypost.com/2025/11/01/us-news/house-re...
68,32,2025-11-02-03-25-50 +0000,nypost,Hero old man saves girl from knife-wielding ma...,https://nypost.com/2025/11/01/world-news/uk-tr...
60,30,2025-11-02-04-29-42 +0000,latimes,"Fireworks light up L.A., Dodgers fans take to ...",https://www.latimes.com/california/story/2025-...
8,29,2025-11-02-11-52-36 +0000,nypost,Two Louvre heist suspects had prior theft conv...,https://nypost.com/2025/11/02/world-news/louvr...
90,26,2025-11-02-00-19-33 +0000,wapo,Trump threatens potential military action in N...,https://www.washingtonpost.com/world/2025/11/0...
97,23,2025-11-01-23-39-02 +0000,bbc,300 million tourists just visited China's stun...,https://www.bbc.com/news/articles/c8x1rnnd5gjo...
81,22,2025-11-02-01-39-48 +0000,nypost,Cuomo praises Adams plan to hire 5K more NYPD ...,https://nypost.com/2025/11/01/us-news/cuomo-pr...
17,21,2025-11-02-11-00-00 +0000,latimes,Barabak: Payback? Power grab? Proposition 50 i...,https://www.latimes.com/politics/story/2025-11...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
